## Moving Average Convergence/Divergence Model (MACD)

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import scipy.stats as ss

# Make plots larger
plt.rcParams['figure.figsize'] = (15, 9)

In [2]:
AAPL = pd.read_csv('AAPL.csv')
AAPL.set_index('Date')
AAPL['OpenTmr'] = AAPL['Open'].shift(-1)
AAPL['OpenClose'] = (AAPL['Open']+ AAPL['Close'])/2
AAPL['HighLow'] = (AAPL['High']+ AAPL['Low'])/2
AAPL['OCHL'] = (AAPL['Open']+ AAPL['Close']+AAPL['High']+ AAPL['Low'])/4

AAPL.head()

,Date,Open,Close,High,Low,Volume,OpenTmr,OpenClose,HighLow,OCHL
0,4-Jan-16,102.610001,105.349998,105.370003,102.000000,67649400,105.750000,103.979999,103.685001,103.832500
1,5-Jan-16,105.750000,102.709999,105.849998,102.410004,55791000,100.559998,104.229999,104.130001,104.180000
2,6-Jan-16,100.559998,100.699997,102.370003,99.870003,68457400,98.680000,100.629998,101.120003,100.875000
3,7-Jan-16,98.680000,96.449997,100.129997,96.430000,81094400,98.550003,97.564999,98.279999,97.922499
4,8-Jan-16,98.550003,96.959999,99.110001,96.760002,70798000,98.970001,97.755001,97.935001,97.845001


In [3]:
N = AAPL.shape[0] # total num days
num_boot = 300# total num bootstrap
T= 250 # start day
window = 200 # training period window

In [4]:
df = AAPL[['Close']]
df.reset_index(level=0, inplace=True)
df.columns=['ds','y']

X1 = df.y.ewm(span=12, adjust=False).mean()
X2 = df.y.ewm(span=26, adjust=False).mean()
X = X1 - X2
Y =AAPL['OpenTmr'][-(N-(T+1)):].values
#X = np.column_stack([np.ones((T,1)),X])
#print(X)
#print(Y.shape)
#movAverage1= (movAvg) 

In [5]:
def MACD(X, Y):
    
    T = X.shape[0]
    #print(T)
    #mu = (AAPL['Open '].mean(),AAPL['Open '].mean(),AAPL['Open '].mean()) 
    #cov = [[1,0.75,-0.35],[0.75,1,0.9],[-0.35,0.9,1]]
    #F = np.random.multivariate_normal(mu,cov,T)
    #Sample for Y,X
    X = np.column_stack([np.ones((T,1)),X])
    #T = X.shape[0]
    N = X.shape
    #beta = np.array([0.56,2.53,2.05,1.78])
    #beta.shape=(N[1],1)
    #Y =X@beta+np.random.normal(0,1,(T,1))
    #Y=AAPL['Open '].values
    #print(T)
    invXX = np.linalg.inv(X.transpose()@X)
    beta_hat = invXX@X.transpose()@Y
    y_hat = X@beta_hat
    
    residuals = Y-y_hat
    sigma2 = (1/T)*residuals.transpose()@residuals

    sigma = np.sqrt(sigma2)

    #variance - covariance of beta_hat
    varcov_beta_hat = (sigma2)*invXX
    std_beta_hat = np.sqrt(T*np.diag(varcov_beta_hat))

    R_square = 1-(residuals.transpose()@residuals)/(T*np.var(Y))

    adj_R_square = 1-(1-R_square)*(T-1)/(T - N[1])

    #Testing Coefficents:beta_i
    #Null Hypotesis

    t_stat = (beta_hat.transpose()-0)/std_beta_hat
    p_val = 1-ss.norm.cdf(t_stat)
    #print(p_val)
    #Test of joint significance
    F_stat= (beta_hat.transpose()@np.linalg.inv(varcov_beta_hat)@beta_hat/N[1])/(residuals.transpose()@residuals/(T-N[1]))
    p_val_F= 1 - ss.f.cdf(F_stat,N[1]-1, T-N[1])
    
    return beta_hat,y_hat

In [6]:
def bootstrap():
    T = 250
    #print(T)
    N = X.shape[0]
    #print(N)
    
    yhat_macd = np.zeros(N-(T+1))
    window = 200
    num_boost = 300 # increase
    
    for t in range(T+1,N):
        X_train = df.y.ewm(span=5, adjust=False).mean()[t-window:t-1]
        #X_train = np.column_stack([np.ones((len(X_train),1)),X_train])
        Y_train = AAPL[['OpenTmr']][t-window:t-1].values
        #print(X_train.shape)
        #print(Y_train.shape)
        X_pred = df.y.ewm(span=5, adjust=False).mean()[t-1:t]
        X_pred = np.column_stack([np.ones((len(X_pred),1)),X_pred])
        
        yhat_train = MACD(X_train , Y_train)[1]
        res_train = Y_train - yhat_train
       
        y_pred_all = np.zeros(num_boost)
       
        for i in range (0,num_boost):
            #err = np.random.choice(res_train,(window-1, ),replace = True)
            err = res_train
            y_bstr = yhat_train + err
            beta_bstr = MACD(X_train,y_bstr)[0]
            #print(X_pred.shape)
            #print(beta_bstr.shape)
 
            y_pred_bstr = X_pred@beta_bstr
            y_pred_all[i] = y_pred_bstr
           
        y_pred_macd = y_pred_all.mean()
       
        yhat_macd[t-(T+1)] = y_pred_macd
   
   
    rmse_macd = np.sqrt(np.mean((Y[27:-1] - yhat_macd[27:-1])**2))
   
    return yhat_macd,rmse_macd

In [7]:
yhat_macd,rmse_macd = bootstrap()

In [8]:
rmse_macd

4.311557789157198

In [9]:
yhat_macd.shape

(627,)

In [11]:
MACD_predict = pd.DataFrame()
MACD_predict['Date'] = AAPL.Date.iloc[-200:]
MACD_predict['Predict_MACD'] = yhat_macd[-200:]

In [12]:
MACD_predict

,Date,Predict_MACD
678,12-Sep-18,224.807784
679,13-Sep-18,224.312747
680,14-Sep-18,225.803518
681,17-Sep-18,225.918260
682,18-Sep-18,223.791370
...,...,...
873,24-Jun-19,196.992658
874,25-Jun-19,197.311029
875,26-Jun-19,196.562120
876,27-Jun-19,197.454737


In [13]:
MACD_predict.to_csv('MACD_prediction.csv')